In [1]:
import praw
import pandas as pd
import numpy as np
import requests
from praw.models import MoreComments
import datetime as dt
from datetime import datetime,timedelta
from psaw import PushshiftAPI
from tqdm import tqdm
import pickle
import dill
import tqdm.notebook as tq
from utils import crawler_utils
from utils.crawler_utils import *


### Initialization

In [3]:
api = PushshiftAPI()
start_time, end_time = time_generation_year(0)
end_time_0 = int(dt.datetime(2021,12,31).timestamp())
start_time_0 = int(dt.datetime(2016,1,1).timestamp())
#start_time, end_time = time_initialization()

In [3]:
print(start_time)

[1609430400, 1612108800, 1614528000, 1617206400, 1619798400, 1622476800, 1625068800, 1627747200, 1630425600, 1633017600, 1635696000, 1638288000]


## Posts Collection

In [ ]:
def collect_one_subreddit_post(subreddit_name,club_abbr):
    temp_list = []
    for i in tq.tqdm(range(len(start_time))):
        temp_list.append(np.array(list(api.search_submissions(after=start_time[i], before=end_time[i],
                                                              subreddit=subreddit_name,
                                                              filter=['url', 'author', 'title', 'subreddit', 'id',
                                                                      'num_comments', 'score', 'ups', 'upvote_ratio','selftext'])
                                       )))
    file = open(f'../data/{club_abbr}.pickle', 'wb')
    dill.dump(temp_list, file)
    file.close()

In [ ]:
def collect_post(club_abbr_list,club_reddit_list):
    for club_index in tq.tqdm(range(len(club_abbr_list))):
        collect_one_subreddit_post(club_reddit_list[club_index],club_abbr_list[club_index])

In [ ]:
api.metadata_.get('shards')

In [ ]:
for club_index in tq.tqdm(range(len(club_abbr))):
    temp_list = []
    for i in tq.tqdm(range(len(start_time))):
        temp_list.append(np.array(list(api.search_submissions(after=start_time[i], before=end_time[i],
                                                              subreddit=club_reddit_abbr[club_index],
                                                              filter=['url', 'author', 'title', 'subreddit', 'id',
                                                                      'num_comments', 'score', 'ups', 'upvote_ratio','selftext'])
                                       )))
    file = open(f'../data/{club_abbr[club_index]}.pickle', 'wb')
    dill.dump(temp_list, file)
    file.close()


### Post Collection in Premier (other)

In [ ]:
collect_post(premier_other_abbr,premier_other_reddit_abbr)

## Reformat the posts

In [ ]:
def reformat_post(club_abbr_list):
    for club_name in tq.tqdm(club_abbr_list):
        filename = f'../data/{club_name}.pickle'
        df = transform_df(filename)
        df = parse_dates(df)
        df.to_csv(f'../data/{club_name}.csv',index=False)

In [ ]:
filename = f'../data/NEW.pickle'
df = transform_df(filename)
df = parse_dates(df)
df.to_csv(f'../data/NEW.csv',index=False)

In [ ]:
reformat_post(club_abbr)

In [ ]:
reformat_post(premier_other_abbr)

## Extracting Comments

### use praw (not recommended, stuck from time to time)

In [ ]:
columns = ['comments','post_id','subreddit','time','title']
reddit = praw.Reddit(client_id='bCnE1U61Wqixgs2wy28POg', client_secret='vEY7k3_j7o3PZZvP-tEt6DnhWr1x5A',
                user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36')
all_comments = pd.DataFrame(columns = columns)
for club_name in tq.tqdm(club_abbr):
    filename = f'../data/{club_name}.csv'    
    all_comments.append(extract_comments(reddit,filename,columns),ignore_index=True)
all_comments.to_csv('../data/all_comments.csv',index=False)

In [ ]:
columns = ['comments','post_id','subreddit','time','title']
reddit = praw.Reddit(client_id='bCnE1U61Wqixgs2wy28POg', client_secret='vEY7k3_j7o3PZZvP-tEt6DnhWr1x5A',
                user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36')
all_comments = pd.DataFrame(columns = columns)
for club_name in tq.tqdm(club_abbr):
    filename = f'../data/{club_name}.csv'
    comments = extract_comments_by_praw(reddit,filename,columns)
    comments.to_csv(f'../data/comments/temp/{club_name}.csv',index=False)
    #all_comments.append(extract_comments(reddit,filename,columns),ignore_index=True)
#all_comments.to_csv('../data/all_comments.csv',index=False)

### use psaw (extract comments one by one, too slow)

In [ ]:
columns = ['comments','post_id','subreddit','time','title']
for club_name in tq.tqdm(club_abbr):
    filename = f'../data/{club_name}.csv'    
    comments = extract_comments_by_psaw(filename,columns)
    comments.to_csv(f'../data/comments/{club_name}.csv',index=False)

### use reddit api (recommended)

### extract comments of posts with over 50 comments

In [ ]:
comments = pd.DataFrame()
columns = ['text','id','post_id','subreddit','time','author','ups']
for club_name in tq.tqdm(club_abbr):
    filename = f'../data/{club_name}.csv'    
    comments = extract_comments_by_api(filename,columns)
    comments.to_csv(f'../data/comments/{club_name}_comments.csv',index=False)

### extract gameday comments (48hrs from the kick-off time)

In [ ]:
comments = pd.DataFrame()
columns = ['text','id','post_id','subreddit','time','author','ups']
for club_name in tq.tqdm(premier_abbr):
    filename = f'../data/{club_name}.csv'
    matchfilename = f'../data/match/{club_name}_match.csv'
    comments = extract_gameday_comments(filename,matchfilename,columns)
    comments.to_csv(f'../data/comments/gameday/{club_name}_gameday_comments.csv',index=False)

In [ ]:
for i in tq.tqdm(range(19,len(premier_abbr))):
    matchfilename = f'../data/match/{premier_abbr[i]}_match.csv'
    subreddit = premier_reddit_abbr[i]
    club_name = premier_abbr[i]
    comments = get_gameday_comments(subreddit,matchfilename)
    comments.to_csv(f'../data/comments/gameday/{club_name}_gameday_comments.csv',index=False)

# Get Comments by player Name

In [ ]:
for name in target_player:
    df = pd.DataFrame()
    for i in tq.tqdm(range(len(start_time))):
        gen = api.search_comments(q = name, before=end_time[i], after=start_time[i], subreddit='ThreeLions',
                                  filter=['author', 'body', 'subreddit', 'id', 'created_utc'])
        results_df = pd.DataFrame({'subreddit': row.subreddit, 'id': row.id, 'time': datetime.fromtimestamp(int(row.created_utc)),
                                   'author': row.author, 'body': row.body} for row in gen)
        df = pd.concat([df, results_df])
    df.to_csv(f'../data/comments/player/{name}_national_team.csv')

In [ ]:
for name in target_player:
    df = pd.DataFrame()
    for i in tq.tqdm(range(len(start_time))):
        gen = api.search_comments(q = name, before=end_time[i], after=start_time[i], subreddit='PremierLeague',
                                  filter=['author', 'body', 'subreddit', 'id', 'created_utc'])
        results_df = pd.DataFrame({'subreddit': row.subreddit, 'id': row.id, 'time': datetime.fromtimestamp(int(row.created_utc)),
                                   'author': row.author, 'body': row.body} for row in gen)
        df = pd.concat([df, results_df])
    df.to_csv(f'../data/comments/player/{name}_Premier.csv')

In [ ]:
for name in target_player:
    df = pd.DataFrame()
    for i in tq.tqdm(range(len(start_time))):
        gen = api.search_comments(q = name, before=end_time[i], after=start_time[i], subreddit='soccer',
                                  filter=['author', 'body', 'subreddit', 'id', 'created_utc'])
        results_df = pd.DataFrame({'subreddit': row.subreddit, 'id': row.id, 'time': datetime.fromtimestamp(int(row.created_utc)),
                                   'author': row.author, 'body': row.body} for row in gen)
        df = pd.concat([df, results_df])
    df.to_csv(f'../data/comments/player/{name}_soccer.csv')

In [ ]:
for name in target_player:
    df = pd.DataFrame()
    for i in tq.tqdm(range(len(start_time))):
        gen = api.search_comments(q = name, before=end_time[i], after=start_time[i], subreddit='football',
                                  filter=['author', 'body', 'subreddit', 'id', 'created_utc'])
        results_df = pd.DataFrame({'subreddit': row.subreddit, 'id': row.id, 'time': datetime.fromtimestamp(int(row.created_utc)),
                                   'author': row.author, 'body': row.body} for row in gen)
        df = pd.concat([df, results_df])
    df.to_csv(f'../data/comments/player/{name}_football.csv')

In [ ]:
for player in range(len(target_player_1)):
    df = pd.DataFrame()
    for i in tq.tqdm(range(len(start_time))):
        gen = api.search_comments(q = target_player_1[player], before=end_time[i], after=start_time[i], subreddit=player_club[player],
                                  filter=['author', 'body', 'subreddit', 'id', 'created_utc'])
        results_df = pd.DataFrame({'subreddit': row.subreddit, 'id': row.id, 'time': datetime.fromtimestamp(int(row.created_utc)),
                                   'author': row.author, 'body': row.body} for row in gen)
        df = pd.concat([df, results_df])
    df.to_csv(f'../data/comments/player/{target_player_1[player]}_{player_club[player]}.csv',index=False)

# Get Comments by Author

In [ ]:
authors = pd.read_csv('../data/author/euro_Kane_soccer_10.csv')
author_list = authors['author']
df = pd.read_csv('../data/comments/player/author/euro_Kane_soccer_author_comments.csv')
#df = pd.DataFrame()
for name in tq.tqdm(author_list):
    for i in tq.tqdm(range(len(start_time))):
        gen = api.search_comments(author = name, before=end_time[i], after=start_time[i],
                                  filter=['author', 'body', 'subreddit', 'id', 'created_utc'])
        results_df = pd.DataFrame({'subreddit': row.subreddit, 'id': row.id, 'time': datetime.fromtimestamp(int(row.created_utc)),
                                   'author': row.author, 'body': row.body} for row in gen)
        df = pd.concat([df, results_df])
    df.to_csv('../data/comments/player/author/euro_Kane_soccer_author_comments.csv',index=False)
df.to_csv('../data/comments/player/author/euro_Kane_soccer_author_comments.csv',index=False)

In [ ]:
authors = pd.read_csv('../data/author/Kane_Premier_author.csv')
author_list = authors['author']
#df = pd.read_csv('../data/comments/player/author/Kane_Premier_author_comments.csv')
df = pd.DataFrame()
for name in tq.tqdm(author_list):
    for i in tq.tqdm(range(len(start_time))):
        gen = api.search_comments(author = name, before=end_time[i], after=start_time[i],
                                  filter=['author', 'body', 'subreddit', 'id', 'created_utc'])
        results_df = pd.DataFrame({'subreddit': row.subreddit, 'id': row.id, 'time': datetime.fromtimestamp(int(row.created_utc)),
                                   'author': row.author, 'body': row.body} for row in gen)
        df = pd.concat([df, results_df])
    df.to_csv('../data/comments/player/author/Kane_Premier_author_comments.csv',index=False)
df.to_csv('../data/comments/player/author/Kane_Premier_author_comments.csv',index=False)

In [ ]:
authors = pd.read_csv('../data/author/Kane_TOT_author.csv')
author_list = authors['author']
df = pd.read_csv('../data/comments/player/author/Kane_TOT_author_comments.csv')
#df = pd.DataFrame()
for name in tq.tqdm(author_list):
    for i in tq.tqdm(range(len(start_time))):
        gen = api.search_comments(author = name, before=end_time[i], after=start_time[i],
                                  filter=['author', 'body', 'subreddit', 'id', 'created_utc'])
        results_df = pd.DataFrame({'subreddit': row.subreddit, 'id': row.id, 'time': datetime.fromtimestamp(int(row.created_utc)),
                                   'author': row.author, 'body': row.body} for row in gen)
        df = pd.concat([df, results_df])
    df.to_csv('../data/comments/player/author/Kane_TOT_author_comments_1.csv',index=False)
df.to_csv('../data/comments/player/author/Kane_TOT_author_comments_1.csv',index=False)

In [ ]:
authors = pd.read_csv('../data/author/LIV_neg_author.csv')
author_list = authors['author']
#df = pd.read_csv('../data/comments/player/author/Kane_TOT_author_comments.csv')
df = pd.DataFrame()
for name in tq.tqdm(author_list):
    for i in tq.tqdm(range(len(start_time))):
        gen = api.search_comments(author = name, before=end_time[i], after=start_time[i],
                                  filter=['author', 'body', 'subreddit', 'id', 'created_utc'])
        results_df = pd.DataFrame({'subreddit': row.subreddit, 'id': row.id, 'time': datetime.fromtimestamp(int(row.created_utc)),
                                   'author': row.author, 'body': row.body} for row in gen)
        df = pd.concat([df, results_df])
    df.to_csv('../data/comments/author_bg/LIV_neg_author_bg.csv',index=False)
df.to_csv('../data/comments/author_bg/LIV_neg_author_bg.csv',index=False)

In [ ]:
authors = pd.read_csv('../data/author/LIV_pos_author.csv')
author_list = authors['author']
#df = pd.read_csv('../data/comments/player/author/Kane_TOT_author_comments.csv')
df = pd.DataFrame()
for name in tq.tqdm(author_list):
    for i in tq.tqdm(range(len(start_time))):
        gen = api.search_comments(author = name, before=end_time[i], after=start_time[i],
                                  filter=['author', 'body', 'subreddit', 'id', 'created_utc'])
        results_df = pd.DataFrame({'subreddit': row.subreddit, 'id': row.id, 'time': datetime.fromtimestamp(int(row.created_utc)),
                                   'author': row.author, 'body': row.body} for row in gen)
        df = pd.concat([df, results_df])
    df.to_csv('../data/comments/author_bg/LIV_pos_author_bg.csv',index=False)
df.to_csv('../data/comments/author_bg/LIV_pos_author_bg.csv',index=False)

In [ ]:
authors = pd.read_csv(f'../data/author/MNU_pos_author.csv')
author_list = authors['author'][10:]
df = pd.read_csv(f'../data/comments/author_bg/MNU_pos_author_bg.csv')
for name in tq.tqdm(author_list):
    for i in tq.tqdm(range(len(start_time))):
        gen = api.search_comments(author = name, before=end_time[i], after=start_time[i],
                                  filter=['author', 'body', 'subreddit', 'id', 'created_utc'])
        results_df = pd.DataFrame({'subreddit': row.subreddit, 'id': row.id, 'time': datetime.fromtimestamp(int(row.created_utc)),
                                   'author': row.author, 'body': row.body} for row in gen)
        df = pd.concat([df, results_df])
    df.to_csv(f'../data/comments/author_bg/MNU_pos_author_bg.csv',index=False)
df.to_csv(f'../data/comments/author_bg/MNU_pos_author_bg.csv',index=False)

In [ ]:
team_list = ['CHE','MNC','TOT']
for team in team_list:
    authors = pd.read_csv(f'../data/author/{team}_neg_author.csv')
    author_list = authors['author']
    #df = pd.read_csv(f'../data/comments/author_bg/{team}_neg_author_bg.csv')
    df = pd.DataFrame()
    for name in tq.tqdm(author_list):
        for i in tq.tqdm(range(len(start_time))):
            gen = api.search_comments(author = name, before=end_time[i], after=start_time[i],
                                      filter=['author', 'body', 'subreddit', 'id', 'created_utc'])
            results_df = pd.DataFrame({'subreddit': row.subreddit, 'id': row.id, 'time': datetime.fromtimestamp(int(row.created_utc)),
                                       'author': row.author, 'body': row.body} for row in gen)
            df = pd.concat([df, results_df])
        df.to_csv(f'../data/comments/author_bg/{team}_neg_author_bg.csv',index=False)
    df.to_csv(f'../data/comments/author_bg/{team}_neg_author_bg.csv',index=False)
    
    authors = pd.read_csv(f'../data/author/{team}_pos_author.csv')
    author_list = authors['author']
    #df = pd.read_csv(f'../data/comments/author_bg/{team}_pos_author_bg.csv')
    df = pd.DataFrame()
    for name in tq.tqdm(author_list):
        for i in tq.tqdm(range(len(start_time))):
            gen = api.search_comments(author = name, before=end_time[i], after=start_time[i],
                                      filter=['author', 'body', 'subreddit', 'id', 'created_utc'])
            results_df = pd.DataFrame({'subreddit': row.subreddit, 'id': row.id, 'time': datetime.fromtimestamp(int(row.created_utc)),
                                       'author': row.author, 'body': row.body} for row in gen)
            df = pd.concat([df, results_df])
        df.to_csv(f'../data/comments/author_bg/{team}_pos_author_bg.csv',index=False)
    df.to_csv(f'../data/comments/author_bg/{team}_pos_author_bg.csv',index=False)

In [ ]:
authors = pd.read_csv(f'../data/author/Sterling_negative_author.csv')
author_list = authors['author']
#df = pd.read_csv(f'../data/comments/author_bg/player_group1_comments.csv')
df = pd.DataFrame()
for name in tq.tqdm(author_list):
    for player in tq.tqdm(player_group1):
        gen = api.search_comments(author = name, q=player,
                                  filter=['author', 'body', 'subreddit', 'id', 'created_utc'])
        results_df = pd.DataFrame({'subreddit': row.subreddit, 'id': row.id, 'time': datetime.fromtimestamp(int(row.created_utc)),
                                   'author': row.author, 'body': row.body, 'player':player} for row in gen)
        df = pd.concat([df, results_df])
    df.to_csv(f'../data/comments/author_bg/player_group1_comments.csv',index=False)
df.to_csv(f'../data/comments/author_bg/player_group1_comments.csv',index=False)

In [4]:
gen = api.search_comments(author = 'aussie_spastic',
                          filter=['author', 'body', 'subreddit', 'id', 'created_utc'])
results_df = pd.DataFrame({'subreddit': row.subreddit, 'id': row.id, 'time': datetime.fromtimestamp(int(row.created_utc)),
                           'author': row.author, 'body': row.body,} for row in gen)
results_df.to_csv(f'../data/vis/author_bg/author2.csv',index=False)

/Users/feegee2000/opt/anaconda3/lib/python3.7/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


# Merge posts

In [ ]:
def merge_posts(club_abbr_list,output_dir):
    all_posts = pd.DataFrame()
    for club_name in club_abbr_list:
        filename = f'../data/{club_name}.csv'
        temp_data = pd.read_csv(filename)
        all_posts = all_posts.append(temp_data)
    all_posts.to_csv(f'../data/{output_dir}.csv',index=False)


In [ ]:
merge_posts(club_abbr,'all_posts')

In [ ]:
merge_posts(premier_abbr,'premier_posts')